In [4]:
import numpy as np
import pandas as pd
import os
import csv, json
import math
import re
from PIL import Image

from collections import Counter

In [5]:
class Stop(Exception):
    pass

class Continue(Exception):
    pass

In [6]:
# main 템플릿

def main(callback, *,
         files=None,
         dirs=None,
         dirs_filter=lambda x: True
         ):
    index:int = 0
    if files:
        for filename in files:
            try:
                callback(index, filename)
            except Continue:
                pass
            except Stop:
                return 0
            finally:
                index += 1
    if dirs:
        for d in dirs:
            for dirpath, _, filenames in os.walk(d):
                for filename in filenames:
                    target = os.path.join(dirpath, filename)
                    if not dirs_filter(target):
                        continue
                    try:
                        callback(index, target)
                    except Continue:
                        pass
                    except Stop:
                        return 0
                    finally:
                        index += 1

In [43]:
OUTPUT_PATH = '../export/scaled'

dirs = [
    '../export/',
]

keywords = [
    'IseSteroids',
    'nvoke',
    'attack',
]

def dirs_filter(target:str):
    if not target.endswith('.csv'):
        return False
    for keyword in keywords:
        return True
        if keyword in target:
            return True
    return False

In [ ]:
def make_image(bs, *, scale_factor=1):
    bs = np.array(bs, dtype=np.uint8)
    img = Image.fromarray(bs, 'L')
    if scale_factor == 1:
        return img
    else:
        return img.resize((img.width * scale_factor, img.height * scale_factor), Image.NEAREST)

def scale_image(dataset:pd.DataFrame, *, identifier, scale_factor=2):
    exportdir = os.path.join(OUTPUT_PATH, f'x{scale_factor}', identifier)
    os.makedirs(exportdir, exist_ok=True)
    for index, row in dataset.iterrows():
        filename, _ = os.path.splitext(row['filename'])
        output = os.path.join(exportdir, f'#{index}_{filename}.png')
        bs = json.loads(row['bytes'])
        pixels = [bs[i:i+16] for i in range(0, len(bs), 16)]
        scaled = make_image(pixels, scale_factor=scale_factor)
        scaled.save(output)
    pass

def proc(index:int, target:str):
    identifier = os.path.splitext(os.path.split(target)[-1])[0]
    dataset = pd.read_csv(target)
    scale_image(dataset, identifier=identifier, scale_factor=1)
    # scale_image(dataset, identifier=identifier, scale_factor=2)
    # scale_image(dataset, identifier=identifier, scale_factor=4)
    # scale_image(dataset, identifier=identifier, scale_factor=8)
    # scale_image(dataset, identifier=identifier, scale_factor=16)

In [45]:
main(proc, dirs=dirs, dirs_filter=dirs_filter)

('1234', '')